In [1]:
import pandas as pd
import psycopg2

In [2]:
!pip install -U textblob
from textblob import TextBlob

Requirement already up-to-date: textblob in c:\users\shiba\anaconda3\lib\site-packages (0.15.3)


In [3]:
connection_string = psycopg2.connect(host="mypostgresdb.cnm9emj4w74o.us-east-2.rds.amazonaws.com", port = 5432, database="project3", user="rootlh", password="password")

In [4]:
df = pd.read_sql_query('select * from post_info', con=connection_string)
df.head()

,type,posts
0,INTJ,'https://www.youtube.com/watch?v=8IEQpfA528M ...
1,INFP,"'It's one of my daily 'music doodles', one min..."
2,INFP,'NO. But I have dyscalculia. :X|||I've learned...
3,ISTP,"'288970 It's been a while, guys. Also I bleach..."
4,INFP,'29 and still have it. Never had a girlfriend....


In [30]:
df['words_per_comment'] = df['posts'].apply(lambda x: len(x.split()))
df['question_per_comment'] = df['posts'].apply(lambda x: x.count('?'))
df['excl_per_comment'] = df['posts'].apply(lambda x: x.count('!'))
df['upper_case'] = df['posts'].str.findall(r'[A-Z]').str.len()
df[['polarity', 'subjectivity']] = df['posts'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
df['emojis'] = df['posts'].str.findall(u'[\U0001f300-\U0001f650]|[\u2000-\u3000]').str.len()
df['ellipsis_per_comment'] = df['posts'].apply(lambda x: x.count('...'))

print(df.head(10))

   type                                              posts  words_per_comment  \
0  INTJ  'https://www.youtube.com/watch?v=8IEQpfA528M  ...               1143   
1  INFP  'It's one of my daily 'music doodles', one min...                873   
2  INFP  'NO. But I have dyscalculia. :X|||I've learned...                665   
3  ISTP  '288970 It's been a while, guys. Also I bleach...                545   
4  INFP  '29 and still have it. Never had a girlfriend....               1399   
5  INFJ  'It gets to be pretty obvious the lifestyle di...               1049   
6  INTP  'I used to find this difficult until I realize...               1190   
7  INTJ  '1. ENTJs - Without fail my favorite people.  ...               1694   
8  ENFP  'Can I just say I'm so glad and thankful that ...               1314   
9  ESFP  'No I'm not, you go home!|||Farnsworth: ENTP F...               1090   

   question_per_comment  excl_per_comment  upper_case  polarity  subjectivity  \
0                    34    

In [31]:
df.describe()

,words_per_comment,question_per_comment,excl_per_comment,upper_case,polarity,subjectivity,emojis,ellipsis_per_comment,I-E,N-S,T-F,J-P
count,8675.000000,8675.000000,8675.000000,8675.000000,8675.000000,8675.000000,8675.000000,8675.000000,8675.000000,8675.000000,8675.000000,8675.000000
mean,1226.241499,10.730951,8.454294,282.125072,0.125152,0.535031,0.462594,33.124265,0.230432,0.137983,0.541095,0.604150
std,311.210544,6.994926,11.610943,94.664190,0.060422,0.045603,2.723027,14.972922,0.421134,0.344902,0.498337,0.489061
min,4.000000,0.000000,0.000000,0.000000,-0.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1046.500000,6.000000,2.000000,224.000000,0.087922,0.505834,0.000000,23.000000,0.000000,0.000000,0.000000,0.000000
50%,1278.000000,10.000000,5.000000,278.000000,0.122788,0.533764,0.000000,33.000000,0.000000,0.000000,1.000000,1.000000
75%,1457.000000,14.000000,11.000000,333.000000,0.159827,0.562812,0.000000,41.000000,0.000000,0.000000,1.000000,1.000000
max,1881.000000,121.000000,219.000000,940.000000,0.558087,1.000000,96.000000,196.000000,1.000000,1.000000,1.000000,1.000000


In [32]:
map1 = {"I": 0, "E": 1}
map2 = {"N": 0, "S": 1}
map3 = {"T": 0, "F": 1}
map4 = {"J": 0, "P": 1}

df['I-E'] = df['type'].astype(str).str[0]
df['I-E'] = df['I-E'].map(map1)

df['N-S'] = df['type'].astype(str).str[1]
df['N-S'] = df['N-S'].map(map2)

df['T-F'] = df['type'].astype(str).str[2]
df['T-F'] = df['T-F'].map(map3)

df['J-P'] = df['type'].astype(str).str[3]
df['J-P'] = df['J-P'].map(map4)

print(df.head(10))

   type                                              posts  words_per_comment  \
0  INTJ  'https://www.youtube.com/watch?v=8IEQpfA528M  ...               1143   
1  INFP  'It's one of my daily 'music doodles', one min...                873   
2  INFP  'NO. But I have dyscalculia. :X|||I've learned...                665   
3  ISTP  '288970 It's been a while, guys. Also I bleach...                545   
4  INFP  '29 and still have it. Never had a girlfriend....               1399   
5  INFJ  'It gets to be pretty obvious the lifestyle di...               1049   
6  INTP  'I used to find this difficult until I realize...               1190   
7  INTJ  '1. ENTJs - Without fail my favorite people.  ...               1694   
8  ENFP  'Can I just say I'm so glad and thankful that ...               1314   
9  ESFP  'No I'm not, you go home!|||Farnsworth: ENTP F...               1090   

   question_per_comment  excl_per_comment  upper_case  polarity  subjectivity  \
0                    34    

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [34]:
X = df.drop(['type','posts','I-E','N-S','T-F','J-P'], axis=1).values

In [35]:
#Introvert vs. Extrovert
y = df['I-E'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)

rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [36]:
IE = rf.score(X_test, y_test)
IE

0.7639465191332411

In [37]:
y_predict = rf.predict(X_test)

In [38]:
accuracy_score(y_test,y_predict)

0.7639465191332411

In [39]:
# Intuitives vs. Sensors
y = df['N-S'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)

rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [40]:
NS = rf.score(X_test, y_test)
NS

0.859843245735362

In [41]:
y_predict = rf.predict(X_test)

In [42]:
accuracy_score(y_test,y_predict)

0.859843245735362

In [43]:
# Thinkers vs. Feelers
y = df['T-F'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)

rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [44]:
TF = rf.score(X_test, y_test)
TF

0.6228676809589673

In [45]:
y_predict = rf.predict(X_test)

In [46]:
accuracy_score(y_test,y_predict)

0.6228676809589673

In [47]:
# Judgers vs. Perceivers
y = df['J-P'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)

rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [48]:
JP = rf.score(X_test, y_test)
JP

0.5629322268326418

In [49]:
y_predict = rf.predict(X_test)

In [50]:
accuracy_score(y_test,y_predict)

0.5629322268326418

In [51]:
# Overall
y = df['type'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)

rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [52]:
overall = rf.score(X_test, y_test)
overall

0.1987090825265099

In [53]:
y_predict = rf.predict(X_test)

In [54]:
accuracy_score(y_test,y_predict)

0.1987090825265099

In [55]:
df.to_csv('Data/data1.csv', index=False)

In [2]:
import numpy as np

In [3]:
df2 = pd.DataFrame(np.array([['Random Forests', 0.7639465191332411, 0.859843245735362, 0.6228676809589673, 0.5629322268326418], ['KNN', 0.716589861751152, 0.8387096774193549, 0.532258064516129, 0.5184331797235023], ['SVM', 0.789, 0.856, 0.539, 0.592], ['Neural Network', 0.7722452878952026, 0.8607653379440308, 0.6445366740226746, 0.6076533198356628], ['Binomial Naive Bayes', 0.7597971415398801, 0.8570769940064545, 0.5606270170585523, 0.5993545412632549]]),
                            columns=['model', 'Introvert/Extrovert Score','Intuitive/Sensing Score', 'Thinking/Feeling Score', 'Judging/Perceiving Score'])

In [4]:
df2

,model,Introvert/Extrovert Score,Intuitive/Sensing Score,Thinking/Feeling Score,Judging/Perceiving Score
0,Random Forests,0.7639465191332411,0.859843245735362,0.6228676809589673,0.5629322268326418
1,KNN,0.716589861751152,0.8387096774193549,0.532258064516129,0.5184331797235023
2,SVM,0.789,0.856,0.539,0.592
3,Neural Network,0.7722452878952026,0.8607653379440308,0.6445366740226746,0.6076533198356628
4,Binomial Naive Bayes,0.7597971415398801,0.8570769940064545,0.5606270170585523,0.5993545412632549


In [5]:
df2.to_csv('Data/scores.csv', index=False)